In [31]:
import matplotlib.pyplot as plt
import numpy as np

import nengo
from nengo.processes import Piecewise
from nengo.processes import WhiteSignal
from nengo.solvers import LstsqL2

In [32]:
model = nengo.Network(label="Test")

with model:
    encoders = np.tile([[1], [1]], (100 // 2, 1))

    def check2(x):
        # print("x0: ", x[0])
        # print("x1: ", x[1])
        if ((x[0] - x[1]) < 0.1) or ((x[1] - x[0]) < 0.1):
            return 1
        else:
            return -1

    inp = Piecewise({0: 1, 1: 0})
    nodeA = nengo.Node(inp)
    ensA = nengo.Ensemble(n_neurons=100, dimensions=1)
    inp2 = Piecewise({0: 1, 0.5: 0})
    nodeB = nengo.Node(inp2)
    ensB = nengo.Ensemble(n_neurons=100, dimensions=1)
    combine = nengo.Ensemble(n_neurons=100, dimensions=2, radius=1.5)
    nengo.Connection(nodeB, ensB)
    nengo.Connection(nodeA, ensA)
    nengo.Connection(ensA, combine[0])
    nengo.Connection(ensB, combine[1])
    out = nengo.Ensemble(n_neurons=100, dimensions=1)
    nengo.Connection(combine, out, function=check2)

    probe1 = nengo.Probe(ensA, synapse=0.01)
    probe2 = nengo.Probe(out, synapse=0.01)
    probe3 = nengo.Probe(ensB, synapse=0.01)

    # length to run simulation (must be long enough to represent each character)
    sim_time = 7
with nengo.Simulator(model) as sim:
    # run for sim_time seconds
    sim.run(sim_time)


plt.figure()
plt.plot(sim.trange(), sim.data[probe1], label="ensA")
plt.plot(sim.trange(), sim.data[probe2], label="Out")
plt.plot(sim.trange(), sim.data[probe3], label="ensB")
plt.legend(loc="upper right")
plt.xlabel("time [s]")
plt.ylabel("Pulse height")
plt.title("Neuron Input")

ValidationError: Ensemble.encoders: shape[1] should be 2 (got 1)